In [2]:
import requests 
import json
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import pickle
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException

In [3]:
# 이쁘게 할 방법이... 어렵다...
def replacer(s):
    s = s.replace('\n', '').replace('\r', '').replace('\t', '')
    return s

In [4]:
# excel, pickle. db에 저장해야 하는뎅... ;ㅅ; 
def save(df, name):
    df.to_excel(name + ".xlsx")
    f = open(name + "_pickle.txt", 'w')
    pickle.dump(df, f)
    f.close()

In [5]:
BASE_URL = "http://store.steampowered.com"
CHART_BASE_URL = "http://steamcharts.com"
SPY_BASE_URL = "http://steamspy.com"

In [10]:
# 스팀차트에서 게임 리스트 크롤링 (세이브 해놓아서 이제 쓸일은 없는듯)
game_df = pd.DataFrame(columns=['name', 'link'])

for i in range(1, 10):
    steamchart_url = CHART_BASE_URL + "/top/p.{page_num}".format(page_num=i)
    response = requests.get(steamchart_url)
    dom = BeautifulSoup(response.content, 'html.parser')
    top_games_table = dom.select("table#top-games td.game-name.left")
    for top_game in top_games_table:
        name = replacer(top_game.select_one('a').text)
        link = top_game.select_one('a').attrs.get('href')
        game_df.loc[len(game_df)] = [name, link]

In [41]:
save(game_df, "game_df_old")

In [6]:
f = open("game_df_pickle.txt")
game_df = pickle.load(f)
f.close()

In [19]:
# 스팀차트에서 게임 플레이어 수 크롤링 (세이브 해놓아서 이제 쓸일은 없는듯2)
player_df = pd.DataFrame(columns=['last', 'median', 'mean'])

for i, row in game_df.iterrows():
    player_list = []
    steamchart_url = BASE_URL + row['link']
    response = requests.get(steamchart_url)
    dom = BeautifulSoup(response.content, 'html.parser')
    player_table = dom.select("div#content-wrapper td.right.num-f")
    for player in player_table:
        player_list.append(float(player.text))
    
    player_df.loc[len(player_df)] = [player_list[-1], np.median(player_list), np.mean(player_list)]


In [ ]:
save(player_df, "player_df")

In [27]:
f = open("player_df_pickle.txt")
player_df = pickle.load(f)
f.close()

In [38]:
driver = webdriver.Firefox()
weighted_tag_df = pd.DataFrame(columns=['name', 'tag', 'weight'])
game_tag_list = []
for i, row in game_df.iterrows():
    steam_url = BASE_URL + row['link']
    driver.get(steam_url)
    
    try:
        select_day = Select(driver.find_element_by_name('ageDay'))
        select_day.select_by_value('25')
        select_month = Select(driver.find_element_by_name('ageMonth'))
        select_month.select_by_value('February')
        select_year = Select(driver.find_element_by_name('ageYear'))
        select_year.select_by_value('1988')
        driver.find_element_by_css_selector("a.btnv6_blue_hoverfade.btn_small").click()
    except NoSuchElementException:
        pass
    
    tag_elements = driver.find_elements_by_css_selector("div.glance_tags.popular_tags a")
    tag_list = []
    i = len(tag_elements)
    for tag in tag_elements:
        if(tag.get_attribute('innerHTML') != u''):
            weighted_tag_df.loc[len(weighted_tag_df)] = [row['name'], replacer(tag.get_attribute('innerHTML')), i/len(tag_elements)]
            tag_list.append(replacer(tag.text))
            i -= 1
    if(len(tag_list) != 0):
        game_tag_list.append(tag_list)
    else:
        game_df = game_df[game_df['name'] != row['name']]
driver.quit()    

In [35]:
not_service_game = ['ARK: Survival Evolved',
'SMITE',
'Football Manager 2015',
'FINAL FANTASY XIV: A Realm Reborn',
'Path of Exile',
'Age of Empires II: HD Edition',
'Spacewar',
'Football Manager 2014',
'TERA',
'Counter-Strike Nexon: Zombies',
'Master of Orion',
'ArcheAge',
'Company of Heroes (New Steam Version)',
'Football Manager 2013',
'Football Manager 2012',
'Age of Mythology: Extended Edition',
'Rocksmith 2014',
'CABAL Online',
'MapleStory',
'FreeStyle2: Street Basketball',
'Dragon Age: Origins - Ultimate Edition']

In [40]:
len(game_tag_list)

204

In [103]:
weighted_tag_df.ix[weighted_tag_df['tag'] == u'','tag'] = u'Early Access'

In [107]:
weighted_tag_df.ix[weighted_tag_df['tag'] == u'Early Access','name']

80                              Rust
325                         Unturned
385           H1Z1: King of the Kill
513                         Factorio
573                        Robocraft
633               H1Z1: Just Survive
653                             DayZ
853                       Brawlhalla
1033               Heroes & Generals
1108                   7 Days to Die
1188                 Space Engineers
1256      Don't Starve Together Beta
1588                  Portal Knights
1638                       Starbound
1658                      The Forest
1718                    Naval Action
1812                      Subnautica
2005                     Creativerse
2203                 Killing Floor 2
2323            Dungeon Defenders II
2557                       Hurtworld
2577                         Nosgoth
2597                 ShellShock Live
2806                  Scrap Mechanic
3013                           Squad
3195                    BeamNG.drive
3351                   Slime Rancher
3

In [108]:
save(weighted_tag_df, 'weighted_tag_df')

In [101]:
f = open("weighted_tag_df_pickle.txt")
weighted_tag_df = pickle.load(f)
f.close()

In [141]:
X = pd.DataFrame(columns=weighted_tag_df["tag"].unique())

In [142]:
X["name"] = game_df["name"]

In [143]:
for i, row in weighted_tag_df.iterrows():
    X[row["tag"]][X["name"] == row["name"]] = row["weight"]

In [144]:
X = X.fillna(0.)

In [145]:
X.describe().T.sort_values("mean", ascending=False)

,count,mean,std,min,25%,50%,75%,max
Multiplayer,204,0.542289,0.332458,0,0.25,0.650000,0.805882,1.000000
Action,204,0.517789,0.355913,0,0.20,0.577778,0.850000,1.000000
Singleplayer,204,0.364671,0.285099,0,0.10,0.300000,0.643908,0.900000
Adventure,204,0.320351,0.319938,0,0.00,0.232143,0.626645,1.000000
Open World,204,0.319224,0.381131,0,0.00,0.050000,0.700000,1.000000
Strategy,204,0.313403,0.378178,0,0.00,0.125000,0.600000,1.000000
Co-op,204,0.269331,0.303782,0,0.00,0.150000,0.500000,1.000000
RPG,204,0.250083,0.362918,0,0.00,0.000000,0.462500,1.000000
Simulation,204,0.241793,0.351960,0,0.00,0.000000,0.500000,1.000000
Free to Play,204,0.237843,0.414492,0,0.00,0.000000,0.200000,1.000000


In [73]:
save(X, 'X')

In [139]:
for row in X.itertuples(index=False):
    print(sum(row[:-1]))

10.5
10.5
10.5
10.5
10.5
10.5
5.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.0
8.5
10.5
10.5
10.5
10.5
4.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
8.0
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
4.5
10.5
10.5
6.5
10.5
10.5
10.5
6.0
10.5
10.5
9.0
10.5
10.5
10.5
10.5
5.5
1.5
10.5
10.5
10.5
7.5
10.5
8.5
10.5
10.5
10.5
10.5
7.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
9.0
10.5
5.5
7.0
10.5
8.5
9.0
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
9.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
6.5
10.5
2.5
10.5
9.5
10.5
10.5
10.5
10.5
10.5
10.5
5.0
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
9.5
5.0
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
10.5
8.5
10.5
10.0
4.0
10.5
10.5
10.5
8.5
10.5
6.0
1.0
10.5
8.0
7.5
10.5
10.0
10.5
5.5
10.5
8.5
10.5
10.5
8.0
10.5
5.5
10.5
10.5
7.5
9.0
10.5
10.5
10.5
10.5
10.5
8.0
10.5
10.5
9.5
5.0
10.5


In [7]:
driver = webdriver.Firefox()
game_top_review_list = pd.DataFrame(columns=["game_name", "sentence"])
for i, row in game_df.iterrows():
    steam_url = BASE_URL + row["link"]
    driver.get(steam_url)
    
    try:
        select_day = Select(driver.find_element_by_name('ageDay'))
        select_day.select_by_value('25')
        select_month = Select(driver.find_element_by_name('ageMonth'))
        select_month.select_by_value('February')
        select_year = Select(driver.find_element_by_name('ageYear'))
        select_year.select_by_value('1988')
        driver.find_element_by_css_selector("a.btnv6_blue_hoverfade.btn_small").click()
    except NoSuchElementException:
        pass
    
    review_elements = driver.find_elements_by_css_selector("div#Reviewsall0 div.content")
    review_sentence_list = []
    for review in review_elements:
        if(review.get_attribute('innerHTML') != u''):
            sentence = []
            for i, row in review.get_attribute('innerHTML'):
                words = str(row).split()
                sentence.append( words )
            review_sentence_list.append(sentence)
driver.quit()   

In [13]:
review_sentence_test = review_test[0].find_all(text=True)

In [21]:
replacer(review_sentence_test[-1]) == u''

True

In [147]:
driver = webdriver.Firefox()

driver.get("http://store.steampowered.com/app/252490/")

try:
    select_day = Select(driver.find_element_by_name('ageDay'))
    select_day.select_by_value('25')
    select_month = Select(driver.find_element_by_name('ageMonth'))
    select_month.select_by_value('February')
    select_year = Select(driver.find_element_by_name('ageYear'))
    select_year.select_by_value('1988')
    driver.find_element_by_css_selector("a.btnv6_blue_hoverfade.btn_small").click()
except NoSuchElementException:
    pass

review_elements = driver.find_elements_by_css_selector("div#Reviewsall0 div.content")
review_elements[0].get_attribute('innerHTML')
#review_sentence_list = []
#for review in review_elements:
#    if(review.get_attribute('innerHTML') != u''):
#        sentence = []
#        for i, row in review.get_attribute('innerHTML'):
#            words = str(row).split()
#            sentence.append( words )
#        review_sentence_list.append(sentence)

<selenium.webdriver.remote.webelement.WebElement (session="bb19dd26-8652-49f2-b6f1-062a4e07749f", element="{f75489c4-adb2-4d4f-b25e-6242824e26b4}")>

In [164]:
review_elements[0].text.split()

[u'I',
 u'start.',
 u'Spawn',
 u'in.',
 u'Nude,',
 u'cold,',
 u'helpless.',
 u'I',
 u'come',
 u'to',
 u'a',
 u'castle',
 u'looking',
 u'structure',
 u'(obviously',
 u'player',
 u'made)',
 u'I',
 u'go',
 u'up',
 u'to',
 u'it',
 u'and',
 u'say',
 u'the',
 u'oblivious',
 u'"Hello!?',
 u"I'm",
 u'friendly"',
 u'Meet',
 u'some',
 u'friendly',
 u'guys',
 u'out',
 u'of',
 u'the',
 u'castle',
 u'They',
 u'inject',
 u'me',
 u'with',
 u'a',
 u'green',
 u'and',
 u'white',
 u'pen',
 u'looking',
 u'thing',
 u'Puts',
 u'my',
 u'health',
 u'to',
 u'100',
 u'I',
 u'get',
 u'hit',
 u'by',
 u'a',
 u'morningstar',
 u'(or',
 u'a',
 u'mace',
 u'if',
 u'you',
 u'want',
 u'me',
 u'to',
 u'be',
 u'correct)',
 u'100HP>20HP',
 u'"You',
 u'are',
 u'our',
 u'slave',
 u'now"',
 u'I',
 u'accept',
 u'my',
 u'duty,',
 u'as',
 u'if',
 u'I',
 u'leave,',
 u'I',
 u'am',
 u'sure',
 u'of',
 u'my',
 u'death',
 u'They',
 u'show',
 u'me',
 u'the',
 u'contents',
 u'of',
 u'a',
 u'box',
 u'outside',
 u'of',
 u'the',
 u'"slave',

In [165]:
driver.quit()  

In [3]:
driver = webdriver.Firefox()

driver.get("http://store.steampowered.com/app/570/")

try:
    select_day = Select(driver.find_element_by_name('ageDay'))
    select_day.select_by_value('25')
    select_month = Select(driver.find_element_by_name('ageMonth'))
    select_month.select_by_value('February')
    select_year = Select(driver.find_element_by_name('ageYear'))
    select_year.select_by_value('1988')
    driver.find_element_by_css_selector("a.btnv6_blue_hoverfade.btn_small").click()
except NoSuchElementException:
    pass



In [12]:
owner_test = driver.find_elements_by_css_selector("div#Reviewsall0 div.persona_name")

In [19]:
for s in owner_test:
    print(s.text)

RevyHuang
Mikewell
VPiGFx
EterNity
potato


In [ ]:
BASE_URL = "http://steamcommunity.com/id/{username}/games/?tab=all"

owner_df = pd.DataFrame(columns=['name', 'game'])
owner_list = []

for name in owner_test:
    owner_url = BASE_URL.format(username=name)
    response = requests.get(owner_url)
    dom = BeautifulSoup(response.content, 'html.parser')
    games_table = dom.select("div#games_list_rows div.gameListRow")
    for game in games_table:
        name = replacer(top_game.select_one('a').text)
        link = top_game.select_one('a').attrs.get('href')
        game_df.loc[len(game_df)] = [name, link]

In [21]:
BASE_URL = "http://steamcommunity.com/id/RevyHuang/games/?tab=all"

owner_df = pd.DataFrame(columns=['name', 'game'])
owner_list = []



In [24]:
driver.get("http://steamcommunity.com/id/RevyHuang/games/?tab=all")
games_test = driver.find_elements_by_css_selector("div#games_list_rows div.gameListRowItemName.ellipsis ")

In [26]:
games_test[0].text

u'Dota 2'